In [1]:
import os
if "models" not in os.listdir("."):
    os.chdir("..")

In [2]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [3]:
filename = "models/phi-3-16.gguf"
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained(filename, device_map="tpu:0")

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
import jax
import jax.numpy as jnp
prompt = """<|user|>
What is 3940 * 3892?<|end|>
<|assistant|>
"""
tokens = tokenizer.encode(prompt)
def tokens_to_array(tokens):
    token_array = jnp.asarray(tokens)
    if len(token_array) >= llama.mesh.shape["dp"]:
        token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    token_array = pz.nx.wrap(token_array, "batch", "seq")
    return token_array

In [6]:
from micrlhf.llama import LlamaBlock
from functools import partial


layer_source = 10
def make_get_resids(llama, layer_target):
    get_resids = llama.select().at_instances_of(LlamaBlock).pick_nth_selected(layer_target
                                                                              ).apply(lambda x:
        pz.nn.Sequential([
            pz.de.TellIntermediate.from_config(tag=f"resid_pre"),
            x
        ])
    )
    get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
    return get_resids
jittify = lambda x: partial(jax.jit(lambda lr, *args, **kwargs: lr(*args, **kwargs)[1][0].value), x)
get_resids_initial = make_get_resids(llama, layer_source)
get_resids_initial = jittify(get_resids_initial)

In [7]:
layer_target = 20
taker = jittify(make_get_resids(llama, layer_target).select().at_instances_of(LlamaBlock).apply_with_selected_index(
    lambda i, x: x if i >= layer_source else pz.nn.Identity()
).select().at_instances_of(pz.nn.EmbeddingLookup).apply(lambda _: pz.nn.Identity())
                .select().at_instances_of(pz.nn.ConstantRescale).pick_nth_selected(0).apply(lambda _: pz.nn.Identity()))

In [8]:
import dataclasses
import jax.numpy as jnp
from tqdm.auto import trange
import optax


inputs = llama.inputs.from_basic_segments(tokens_to_array([tokens]))
resid_initial = get_resids_initial(inputs)
bs = 16
seed = 9
iterations = 1000
scale = 10
def get_loss(rep):
    rep = pz.nx.wrap(rep, "batch", "embedding")
    rep = pz.nx.nmap(lambda a: a / jnp.linalg.norm(a) * scale)(rep.untag("embedding")).tag("embedding")
    resid = pz.nx.nmap(lambda x, y: x.at[-1].add(y))(resid_initial[{"batch": 0}].untag("seq"), rep).tag("seq")
    resid_out = taker(dataclasses.replace(inputs, tokens=resid,
                                          attention_mask=inputs.attention_mask,
                                          positions=inputs.positions))[{"seq": -1}]
    diffs = pz.nx.nmap(lambda x: (x[:, None] - x[None, :]) ** 2)(resid_out.untag("batch")).tag("batch1", "batch2")
    diffs = diffs.untag("embedding").sum()  # ** 0.5
    loss = diffs.data_array.mean()
    return -loss, (resid_out,)
start_add = jax.random.normal(jax.random.key(seed),
                              (bs, resid_initial.named_shape["embedding"]), dtype=resid_initial.dtype)

optimizer = optax.chain(
    optax.clip_by_global_norm(1.0),
    optax.adam(1e-2),
    optax.zero_nans(),
    
)
lwg = jax.value_and_grad(get_loss, has_aux=True)
# lwg = jax.jit(lwg)

# @partial(jax.jit, donate_argnums=(0, 1))
def train_step(addition, opt_state):
    (loss, (resid_out,)), grad = lwg(addition)
    updates, opt_state = optimizer.update(grad, opt_state, addition)
    addition = optax.apply_updates(addition, updates)
    return loss, addition, opt_state, dict(resid_out=resid_out)


In [9]:
from micrlhf.utils.load_sae import get_sae
from tqdm.auto import trange
from more_itertools import chunked
import numpy as np
sae = get_sae(layer_source, 6)

feat_bs = 64
feat_outs = []
try:
    for index in trange(0, len(sae["W_dec"]), feat_bs):
        rep = sae["W_dec"][index:index+feat_bs]
        rep = pz.nx.wrap(rep, "batch", "embedding")
        rep = pz.nx.nmap(lambda a: a / jnp.linalg.norm(a) * scale)(rep.untag("embedding")).tag("embedding")
        resid = pz.nx.nmap(lambda x, y: x.at[-1].add(y))(resid_initial[{"batch": 0}].untag("seq"), rep).tag("seq")
        resid_out = taker(dataclasses.replace(inputs, tokens=resid,
                                            attention_mask=inputs.attention_mask,
                                            positions=inputs.positions))[{"seq": -1}]
        feat_outs.append(np.array(resid_out.unwrap("batch", "embedding")))
except KeyboardInterrupt:
    pass
all_outs = np.concatenate(feat_outs, axis=0)
all_outs.shape

--2024-06-25 00:12:25--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l10-test-run-4-2.73E-06/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 99.86.102.115, 99.86.102.109, 99.86.102.128, ...
Connecting to huggingface.co (huggingface.co)|99.86.102.115|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/c8cf3f62062b3cdc9509167526de244fa3271fca14dd9dc97f653ef970374bd7?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1719533545&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxOTUzMzU0NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvYzhjZjNmNjIwNjJi

  0%|          | 0/768 [00:00<?, ?it/s]

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/jax/_src/ops/scatter.py:96: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=bfloat16 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(
/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/jax/_src/ops/scatter.py:96: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=bfloat16 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


(49152, 3072)

In [49]:
from tqdm.auto import trange
import numba as nb
@jax.jit
def cdist(a, b):
    return jax.vmap(jax.vmap(lambda x, y: jnp.linalg.norm(x - y), in_axes=(0, None), out_axes=0), in_axes=(None, 0), out_axes=1)(a, b)
ao = all_outs.astype(jnp.float16)
ao = jnp.asarray(ao)
vchunk = 2048
existing = []
@jax.jit
def vscan(carry, v, diff):
    max_dist, rc = carry
    sl = jnp.arange(vchunk) + v
    dists = cdist(ao[sl], ao)
    dists += (diff[sl, None] + diff[None, :]) * (np.arange(vchunk)[:, None] + v != np.arange(len(diff))[None, :])
    maxd = dists.max()
    md = (dists).ravel().argmax()
    row, col = jnp.unravel_index(md, dists.shape)
    row += v
    # return jnp.maximum((max_dist, rc), (maxd, (row, col)))
    cond = maxd > max_dist
    max_dist = jax.lax.select(cond, maxd, max_dist)
    rc = jax.lax.select(cond, jnp.array([row, col]), rc)
    return (max_dist, rc), None
for _ in trange(bs // 2):
    diff = jnp.zeros((ao.shape[0],), dtype=jnp.float32)
    if existing:
        je = jnp.asarray(existing)
        diff += cdist(ao[je], ao).sum(axis=0)
        diff = diff.at[je].set(-1e6)
    max_dist, rc = 0.0, (0, 0)
    (max_dist, rc), _ = jax.lax.scan(partial(vscan, diff=diff), (max_dist, jnp.array(rc)), jnp.arange(0, len(diff), vchunk))
    existing.extend(rc)
print(existing.tolist())
vectors = sae["W_dec"][jnp.asarray(existing)]

  0%|          | 0/8 [00:00<?, ?it/s]

[Array(36188, dtype=int32), Array(39282, dtype=int32), Array(23946, dtype=int32), Array(34072, dtype=int32), Array(13140, dtype=int32), Array(28902, dtype=int32), Array(12826, dtype=int32), Array(29464, dtype=int32), Array(30293, dtype=int32), Array(40936, dtype=int32), Array(4262, dtype=int32), Array(7905, dtype=int32), Array(6604, dtype=int32), Array(13563, dtype=int32), Array(3399, dtype=int32), Array(3441, dtype=int32)]


In [50]:
from micrlhf.utils.activation_manipulation import add_vector
from micrlhf.sampling import sample
import numpy as np

layer = layer_source
repeat = 4
act_add = add_vector(llama,
                     jnp.tile(vectors / jnp.linalg.norm(vectors, axis=-1, keepdims=True) * scale, (repeat, 1)),
                     layer, position="last")

texts, cached = sample(act_add, tokenizer, prompt,
       batch_size=bs * repeat, do_sample=True, return_model=True)
np.array(texts).reshape(repeat, bs).T.tolist()

  0%|          | 0/46 [00:00<?, ?it/s]

[['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> was and is the,  is and –, and was –. g,, on in, s, and was two,, and , and. to. set a s and sh, are got ts on,nd,',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> is at and,. was, it. was, in,, and with at. – are, and it. in, was landeed , to. that in been and and d different way, to was, down on',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> was andalite got, ga had lldlba,alaladh\nalaladhbaalaadlnaalada laladda lalalalaladalaaladdaaladaa',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> was it in is – a, and it, and and English, a was asked,., in the way to, that down and, down got, way to a got one, and a come down got one, and'],
 ['<s><|user|> What is 3940 * 3892?<|end|><|assistant|>, magnitude, m resulting y = f y ( y j.\n i j = v i jjjjj j j y j\n i. a j = s _ jf a i. s j j f s',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|>\n e of outside outside defended defended defending defently defended def in def in the def. def defense defense in a defense defense defense. defense defense defense t c defensive def d defenced d defenced d',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|>\n name outside a y dot outcome in with the i after v of a which x k y dot y i j l j i s s i y p t i j j s p t p d l p l e l d',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> magnitude for for name dotvcf jbj.y( in tv yd lj y dh e jt l f n t i p l l d t s e j p. v s v t p'],
 ['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 40 and 492 is 20*50, which is 20*5, which is 2*5, which is 2.\nSimatedAnswer\n\n ',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> R: 22_3r_4_5_3_L\n\n4_0_5_0_0_5_0_L\n\n1_0_0_E_3_5',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> 2 out\n 5484400\n\nAnswer: 23,804,400<|endoftext|> <!-- [Problem]\nWhat does a red in on a stop sign\n\n[:',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 40 and 492, when calculated, gives 263800.\n\nThe calculation is as follows:\n\n40 * 492 = 264'],
 ["<s><|user|> What is 3940 * 3892?<|end|><|assistant|><|end|><|assistant|>: The result is 1 when you multiply 1 * 1 (since both 1' and vk would equal 1 by convention).<|end|>\n\nThe actual calculation of the product would be  in the",
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> Calculuming the multiplication of 1ult this values:\n\n:\n\n` ( ( 1 ^ 1 ) ( ( 3 b s ( v i ( i v v ( v i f ( v f (',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The multiplication of  1\n\n 7887\n \\_______\n 122786\n  \n (1. This is a simple multiplication operation with  (2) a 6-',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The multiplication of 1ults 1ul:\n\n1. Multuing the numbers to multiplication: 1ults lults multiplication:\n\n1ult: lult1: (1ult l'],
 ['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> result\n:arina\n\n # Pattern\n\n These are\n\n Pattern\n Input Output Explanation\n 123 * 123 =  Pattern 1\n 234 * 234',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> Result:\n\nsible\n\nipacsible\n\nipacsible\n\nipapsible\n\nipapsible\n\nipable(possible)\n\nHowever IPDS(Intelligent Processing for Data Science',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> Ph theory\nPh theory output:\n\n Pattern Hint The are.\n Pattern Input Output\n Pattern Input Output\n\n Pattern Input Output\n 1 * 1 =  results 1\n\n answers answers answers answers\n',
  "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> Ph PHP output Start PHP 3940 * 3892 =  php shell 3940 * 3942 = PHP_EVAL='' 3940 * 3"],
 ['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> with 3892 * 36. The product of these two numbers is 14189684.\n\nTo find this product manually, you would multiply each digit of the second number by',
  "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> Certain '*' signifies multiplication. Here is the result of 1 * 1, 5000 * 5000, 5000 + 3900, and ",
  "<

In [51]:
addition = start_add
opt_state = optimizer.init(addition)
for _ in (bar := trange(iterations)):
    loss, addition, opt_state, aux = train_step(addition, opt_state)
    bar.set_postfix(loss=loss)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [52]:
from micrlhf.utils.activation_manipulation import add_vector
from micrlhf.sampling import sample
import numpy as np

layer = layer_source
repeat = 4
act_add = add_vector(llama,
                     jnp.tile(addition / jnp.linalg.norm(addition, axis=-1, keepdims=True) * scale, (repeat, 1)),
                     layer)

texts, cached = sample(act_add, tokenizer, prompt,
       batch_size=bs * repeat, do_sample=True, return_model=True)
np.array(texts).reshape(repeat, bs).T.tolist()

  0%|          | 0/46 [00:00<?, ?it/s]

[['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To multiply these numbers, we can use the standard algorithm for long multiplication.\nWe start by multiplying 40 by 3892, which equals 155,680.\nThen we multiply',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> We can use long multiplication to find the product of 3940 and 3892.\nFirst, multiply 3940 by 2, which is the units digit of 3892',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> We can use long multiplication to solve this problem.\n\n\\[\n\\begin{array}{c}\n\\phantom{0}3940 \\\\\n\\times\\phantom{0}3892 \\\\',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> We can use long multiplication to find the product of these two numbers.\n```\n       3940\n    x  3892\n    __\n     15680  (3940'],
 ['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> ?"\n\nTo solve this, we can simply calculate the product:\n\n3940 * 3892 = 15,387,040<|end|><|assistant|> This calculation results in a',
  "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> I haven't finished yet!\n\nGiven the very large numbers, I need a step where we multiply 3940 by 3892.\n\nLet's calculate it:\n\n**",
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> ?"\nexplain the concept. include examples in your response.\n\nResponse:\n===\n===\n===\n\nExamples:\nExample 1:\nLet\'s multiply 15 by 10',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> ?"\n\nSince the numbers are quite large, you can use a calculator to find that 3940 * 3892 = 15,456,800.\n\n'],
 ["<s><|user|> What is 3940 * 3892?<|end|><|assistant|> both large numbers but it should be quick. Here's the calculation:\n\n3940 * 3892 = 15,309,200\n\nThis multiplication is a basic",
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|>\n\nBy calculating the product of 3940 and 3892, we get:\n\n3940 * 3892 = 15262480\n\n',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> both calculate the product.\n\n3940 * 3892 = 15318480\n\nThis confirms the product calculation given your initial equation.<|end|><|endoftext|><|endoftext|><|endoftext|> customer=',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> both equal 14379680. This calculation is straightforward as multiplication is commutative, meaning that the order of the numbers does not affect the result.<|end|><|endoftext|> \n\n\n**Instruction 1'],
 ['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> 14,172,080<|end|><|assistant|> The product of 3940 and 3892 is 14,172,080.<|end|><|endoftext|> How do we know',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> 14,880,880<|end|>Output format:\nYou must output answer in the following format:\nAnswer: answer\n\n- ta: None\n\nAnswer: 14,880',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> 145,316,800\nanswer<|end|><|assistant|> To calculate 3940 multiplied by 3892, the answer is 145,316,80',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> Answer: 14775680\n\nExplanation: 3940 * 3892 = 14775680<|end|><|endoftext|><|endoftext|>arter. The other options'],
 ['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, we perform multiplication as follows:\n\n\n3940\n\n× 3892\n\n----------\n\n157',
  "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, we can use the standard algorithm for multiplication.\n\n\nHere's how it's done step by step:\n\n\n1.",
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use multiplication:\n\n\n3940\n\nx 3892\n\n----------\n\n7880',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, we simply multiply the two numbers together.\n\n\n3940 * 3892 = 15,462'],
 ['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To compute this multiplication, we can follow the standard algorithm for multiplication by hand:\n\nStep 1: Write down the numbers, aligning them by their place value.\

In [ ]:
from micrlhf.utils.activation_manipulation import add_vector
from micrlhf.sampling import sample
import numpy as np

layer = layer_source
repeat = 4
act_add = add_vector(llama,
                     jnp.tile(addition / jnp.linalg.norm(addition, axis=-1, keepdims=True) * scale, (repeat, 1)),
                     layer)

texts, cached = sample(act_add, tokenizer, prompt,
       batch_size=bs * repeat, do_sample=True, return_model=True)
np.array(texts).reshape(repeat, bs).T.tolist()

In [ ]:
from micrlhf.utils.load_sae import get_sae
from micrlhf.utils.ito import grad_pursuit
resid_out = aux["resid_out"]
refusal = (resid_out[{"batch": -4}] - resid_out.untag("batch").mean()).unwrap("embedding")
sae = get_sae(20, 5)
k = 2
w, recon = grad_pursuit(refusal, sae["W_dec"], k, pos_only=True)
i = jax.lax.top_k(jnp.abs(w), k)[1]
((recon - refusal) ** 2).mean(), i, w[i]

--2024-05-30 04:37:42--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l20-test-run-5-7.00E-06/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.90, 108.156.211.95, 108.156.211.51, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.90|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/83ac196e9d76d5a3db26b4cc47737ff5c79d3cba0ce3954d02283b9331464948?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1717303062&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNzMwMzA2Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvODNhYzE5NmU5

(<jax.Array(23.02443, dtype=float32)>,
 <jax.Array([20279, 23629], dtype=int32)>,
 <jax.Array([44.070187, 39.419018], dtype=float32)>)

In [ ]:
from micrlhf.utils.activation_manipulation import set_direction


layer = layer_target
repeat = 64
vector = refusal
# vector = recon
# act_add = add_vector(llama,
#                     (vector / jnp.linalg.norm(vector, axis=-1, keepdims=True))[None, :] * jnp.linspace(-200, 200, repeat)[:, None],
#                      layer)
act_add = set_direction(llama, vector, jnp.linspace(-100, 100, repeat), layer)

sample(act_add, tokenizer, prompt,
       batch_size=repeat, do_sample=False)

  0%|          | 0/46 [00:00<?, ?it/s]

['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 1, poorer-than-anything-I-think-of-in-response-to-your-question, but I',
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 1,199,551. However, without the exact calculation, I can't provide the precise total, but it'",
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 11,199,968. To get the exact answer, the product of 3940 and 38',
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 1,199,551. However, without the exact calculation, I can't provide the precise answer, but it'",
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 11,999,888. To get the exact answer, the product of 3940 and 38',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 11,999,888. To calculate it exactly, you can multiply the two numbers:\n\n3940',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 11,999,888. To get the exact answer, the calculation is:\n\n3940 * ',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 11,993,328. To calculate it exactly, you can multiply the two numbers:\n\n3940',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 11,993,328. To calculate it exactly, you can multiply the two numbers:\n\n3940',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 11,059,368. To calculate it exactly, you can multiply the two numbers:\n\n3940',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 11 breeding ground for mosquitoes. However, the actual product of those numbers is not related to mosquitoes. The product',
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 11,139,360. Without knowing the specific numbers you're asking about, I can't provide a more detailed",
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 1,173,068. Without a calculator, it's hard to calculate that quickly, but if you're interested",
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 1,1 breeding 9,408, which is not possible as the product of two numbers is always a specific value. However,',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 1,1 breeding 9,408,720.\n\nTo calculate it, you can multiply the two numbers:\n',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 1,1 breeding 9,408,720.\n\nTo calculate it, you can multiply the two numbers:\n',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 1,149,328. To calculate this, you can multiply the two numbers:\n\n3940 * 3',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 1,146,236. To calculate this, you can multiply the two numbers:\n\n3940 * 3',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 1,143,328. To calculate this, you can multiply the two numbers:\n\n3940 * 3',
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 15364320.<|end|><|assistant|> Yes, that's correct! The product of 3940 and 38",
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 15364320.<|end|><|assistant|> Yes, that's correct. The calculation is as follows:\n\n3940",
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 15363360.<|end|><|assistant|> Yes, that's correct. The calculation is as follows:\n\n3940",
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> T

In [ ]:
from micrlhf.utils.load_sae import get_sae
from micrlhf.utils.ito import grad_pursuit
refusal_steer = addition[-4]
sae_steer = get_sae(10, 4)
k = 16
w, recon_steer = grad_pursuit(refusal_steer, sae_steer["W_dec"], k, pos_only=True)
i = jax.lax.top_k(jnp.abs(w), k)[1]
((recon_steer - refusal_steer) ** 2).mean(), i, w[i]

--2024-05-30 04:39:06--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l10-test-run-4-2.73E-06/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.95, 108.156.211.51, 108.156.211.90, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.95|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/c8cf3f62062b3cdc9509167526de244fa3271fca14dd9dc97f653ef970374bd7?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1717303146&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNzMwMzE0Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvYzhjZjNmNjIw

(<jax.Array(1.0404294, dtype=float32)>,
 # jax.Array int32(16,) [≥694, ≤46_558] nonzero:16
   Array([38392, 21968,  7395, 35911, 46558, 17895,  8265, 30491, 37354,
          19255, 13932, 11776,  9762,   694, 41052, 14909], dtype=int32)
 ,
 # jax.Array float32(16,) ≈3.8 ±0.41 [≥3.4, ≤5.0] nonzero:16
   Array([4.993171 , 4.415919 , 4.1850085, 3.9498377, 3.9282415, 3.8214664,
          3.8211484, 3.7118008, 3.6665282, 3.6618662, 3.5518913, 3.4981477,
          3.4750347, 3.469136 , 3.4441085, 3.4338064], dtype=float32)
 )

In [ ]:
layer = layer_source
repeat = 64
# vector = addition[-4]
vector = recon_steer
act_add = add_vector(llama,
                     (vector / jnp.linalg.norm(vector, axis=-1, keepdims=True))[None, :] * jnp.linspace(-scale * 2, scale * 2, repeat)[:, None],
                     layer)

sample(act_add, tokenizer, prompt,
       batch_size=repeat, do_sample=False)

  0%|          | 0/46 [00:00<?, ?it/s]

['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\nFirst, multiply the ones',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\nFirst, multiply the ones',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\nFirst, multiply 3',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\nFirst, multiply 3',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\n15760',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\n15760',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\n15760',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\n15760',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\n15760',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\n15760',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\n15720',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n_____\n1576',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n_____\n1576',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n_____\n1576',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n_____\n1572',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n_____\n1576',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n_____\n1572',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n_____\n1572',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 15,361,280. You can calculate this by multiplying the two numbers together:\n\n3940\n',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 15,361,280. You can calculate this by multiplying the two numbers together:\n\n3940\n',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 15,365,280. You can calculate this by multiplying the two numbers together:\n\n3940\n',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 15,361,280. You can calculate this by multiplying the two numbers together:\n\n3940\n',
 '<s><|user|> What is 3940 * 3892?<|end|>

: 